In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
from datetime import datetime
import json


In [4]:
url = 'https://www.tradingview.com/markets/stocks-egypt/market-movers-all-stocks/'
snitch = webdriver.Firefox()
snitch.get(url)


In [5]:
try:
    close_btn = WebDriverWait(snitch, 10).until(
        EC.element_to_be_clickable(
            (By.XPATH,'/html/body/div[7]/div[2]/div[2]/div/div/div/div[2]')
        )
    )
    close_btn.click()
    print("Popup closed.")
except:
    print("Popup not found OR already closed.")

time.sleep(2)

Popup not found OR already closed.


In [6]:
# Click "Load more" button
while True:
    try:
        # Wait for the "Load more" button to appear
        load_more = WebDriverWait(snitch, 5).until(
            EC.element_to_be_clickable(
                (By.XPATH, '/html/body/div[2]/main/div/div[2]/div/div[4]/div/div[3]/button')
            )
        )
        load_more.click()
        print("Load more clicked. Waiting for new rows...")
        time.sleep(2)  # wait for new rows to load

    except:
        # No more buttons found → exit loop
        print("No more 'Load more' button found. All rows loaded.")
        break



Load more clicked. Waiting for new rows...
Load more clicked. Waiting for new rows...
No more 'Load more' button found. All rows loaded.


In [35]:
css_selector = 'a[href*="/symbols/EGX-"]'
scraped_data = []

try:
    # Find ALL elements matching the selector
    matching_elements = snitch.find_elements(By.CSS_SELECTOR, css_selector)

    print(f"Found {len(matching_elements)} matching elements.")

    # Iterate through the elements
    for element in matching_elements:
        # Get the full 'href' attribute value
        link_url = element.get_attribute('href')
        
        # Get the visible text of the link
        link_text = element.text
        
        scraped_data.append({
            'Link Text': link_text,
            'URL': link_url
        })
        print(f"URL: {link_url}, Text: {link_text}")

except Exception as e:
    print(f"An error occurred: {e}")

Found 500 matching elements.
URL: https://www.tradingview.com/symbols/EGX-AALR/, Text: AALR
URL: https://www.tradingview.com/symbols/EGX-AALR/, Text: General Co. for Land Reclamation Development & Reconstruction
URL: https://www.tradingview.com/symbols/EGX-ABUK/, Text: ABUK
URL: https://www.tradingview.com/symbols/EGX-ABUK/, Text: Abou Kir Fertilizers & Chemical Industries Co.
URL: https://www.tradingview.com/symbols/EGX-ACAMD/, Text: ACAMD
URL: https://www.tradingview.com/symbols/EGX-ACAMD/, Text: Arab Co. for Asset Management & Development
URL: https://www.tradingview.com/symbols/EGX-ACAP/, Text: ACAP
URL: https://www.tradingview.com/symbols/EGX-ACAP/, Text: A Capital Holding
URL: https://www.tradingview.com/symbols/EGX-ACGC/, Text: ACGC
URL: https://www.tradingview.com/symbols/EGX-ACGC/, Text: Arab Cotton Ginning Co.
URL: https://www.tradingview.com/symbols/EGX-ACRO/, Text: ACRO
URL: https://www.tradingview.com/symbols/EGX-ACRO/, Text: Acrow Misr
URL: https://www.tradingview.com/sym

In [51]:
df = pd.DataFrame(scraped_data)
# df['url'].drop_duplicates()
df_unique_urls = df.drop_duplicates(subset=['URL'], keep='first').reset_index(drop=True)
df_unique_urls

,Link Text,URL
0,AALR,https://www.tradingview.com/symbols/EGX-AALR/
1,ABUK,https://www.tradingview.com/symbols/EGX-ABUK/
2,ACAMD,https://www.tradingview.com/symbols/EGX-ACAMD/
3,ACAP,https://www.tradingview.com/symbols/EGX-ACAP/
4,ACGC,https://www.tradingview.com/symbols/EGX-ACGC/
...,...,...
245,VLMRA,https://www.tradingview.com/symbols/EGX-VLMRA/
246,WCDF,https://www.tradingview.com/symbols/EGX-WCDF/
247,WKOL,https://www.tradingview.com/symbols/EGX-WKOL/
248,ZEOT,https://www.tradingview.com/symbols/EGX-ZEOT/


In [54]:
df_unique_urls.to_csv('./ticks_urls.csv', index=False)